
### Extrapolation with text-davinci-002 (Rationales+Markups)

In [17]:
!pip install --upgrade openai

     |████████████████████████████████| 42 kB 1.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 7.4 MB/s 
  Created wheel for openai: filename=openai-0.18.1-py3-none-any.whl size=53168 sha256=4fd30f991e4978b80b71e22657f1763d1bb5dd3911e6bfa4b5e1658381d61d3d
  Stored in directory: /root/.cache/pip/wheels/5a/bf/24/fcdc9d2b81f9c7e565bb2036ec9f7cc930056b829895b3bf48
Successfully built openai


In [4]:
#path of the TSV file to be saved
SAVE_PATH = ''
%env SAVE_PATH=$SAVE_PATH

DATA_PATH = ''
%env DATA_PATH=$DATA_PATH

%env OPENAI_API_KEY=

env: SAVE_PATH=
env: DATA_PATH=
env: OPENAI_API_KEY=


## utils

In [3]:
import string
import pandas as pd
import itertools
import random
from collections import Counter
import matplotlib.pyplot as plt
import json

def save_results(prompt, ground_truth, responses_, file_name):
  
  results = {'prompt':[], 'ground_truth':[],'candidate':[]}
  for p, gt, resp in zip(prompt, ground_truth, responses_):
    results['prompt'].append(p)
    results['ground_truth'].append(gt)
    results['candidate'].append(resp['choices'][0]['text'])

  #save results
  df = pd.DataFrame(results)
  df.to_csv(file_name, sep='\t', index = False)

## Prompt

In [18]:
base_prompt =  "Example 1:\nInstruction: add the numbers E 4 D 1 C 6 B 5 A 4 and D 1 C 8 B 5 A 3   \nExplanation:\nThis first number goes until position E, whereas the second goes until position D. The first number is the longest, so we sum digits in positions A, B, C, D, E.\nWe sum the digits in position A, which are A 4 and A 3. That is 4 + 3, which equals to 7. Because 7 is less than or equal to 9, this results in carry 0 and A 7.\nThis carry 0 is summed with digits in position B, which are B 5 and B 5. That is 0 + 5 + 5, which equals to 10. Because 1 0 is greater than 9, this results in carry 1 and B 0.\nThis carry 1 is summed with digits in position C, which are C 6 and C 8. That is 1 + 6 + 8, which equals to 15. Because 1 5 is greater than 9, this results in carry 1 and C 5.\nThis carry 1 is summed with digits in position D, which are D 1 and D 1 .That is 1 + 1 + 1, which equals to 3. Because 3 is less or equal than 9, this results in carry 0 and D 3.\nThis carry 0 is summed with digits in position E, which are E 4 and E 0. That is 0 + 4 + 0, which equals to 4. Because 4 is less than or equal to 9, this results in carry 0 and E 4. \nOutput: E 4 D 3 C 5 B 0 A 7\n\nExample 2:\nInstruction: add the numbers C 7 B 8 A 0 and D 2 C 3 B 0 A 8\nExplanation:\nThis first number goes until position C, whereas the second goes until position D. The second number is the longest, so we sum digits in positions A, B, C, D.\nWe sum the digits in position A, which are A 0 and A 8. That is 0 + 8, which equals to 8. Because 8 is less than or equal to 9, this results in carry 0 and A 8.\nThis carry 0 is summed with digits in position B, which are B 8 and B 0. That is 0 + 8 + 0, which equals to 8. Because 8 is less than or equal to 9, this results in carry 0 and B 8.\nThis carry 0 is summed with digits in position C, which are C 7 and C 3. That is 0 + 7 + 3, which equals to 10. Because 1 0 is greater than 9, this results in carry 1 and C 0.\nThis carry 1 is summed with digits in position D, which are D 0 and D 2. That is 1 + 0 + 2, which equals to 3. Because 3 is less than or equal to 9, this results in carry 0 and D 3.\nOutput: D 3 C 0 B 8 A 8\n\nExample 3:\nInstruction: add the numbers C 9 B 6 A 4 and C 9 B 4 A 2\nExplanation:\nThis first number goes until position C, whereas the second goes until position C. Both numbers have equal length, so we sum digits in positions A, B, C.\nWe sum the digits in position A, which are A 4 and A 2. That is 4 + 2, which equals to 6. Because 6 is less than or equal to 9, this results in carry 0 and A 6.\nThis carry 0 is summed with digits in position B, which are B 6 and B 4. That is 0 + 6 + 4, which equals to 10. Because 1 0 is greater than 9, this results in carry 1 and B 0.\nThis carry 1 is summed with digits in position C, which are C 9 and C 9. That is 1 + 9 + 9, which equals to 19. Because 1 9 is greater than 9, this results in carry 1 and C 9.\nThis remaining carry results in D 1.\nOutput: D 1 C 9 B 0 A 6\n\nExample 4:\nInstruction: add the numbers D 8 C 7 B 6 A 2 and D 6 C 3 B 7 A 2\nExplanation:\nThis first number goes until position D, whereas the second goes until position D. Both numbers have equal length, so we sum digits in positions A, B, C, D.\nWe sum the digits in position A, which are A 2 and A 2. That is 2 + 2, which equals to 4. Because 4 is less than or equal to 9, this results in carry 0 and A 4.\nThis carry 0 is summed with digits in position B, which are B 6 and B 7. That is 0 + 6 + 7, which equals to 13. Because 1 3 is greater than 9, this results in carry 1 and B 3.\nThis carry 1 is summed with digits in position C, which are C 7 and C 3. That is 1 + 7 + 3, which equals to 11. Because 1 1 is greater than 9, this results in carry 1 and C 1.\nThis carry 1 is summed with digits in position D, which are D 8 and D 6. That is 1 + 8 + 6, which equals to 15. Because 1 5 is greater than 9, this results in carry 1 and D 5.\nThis remaining carry results in E 1.\nOutput: E 1 D 5 C 1 B 3 A 4\n\nExample 5:\nInstruction: add the numbers D 8 C 0 B 6 A 2 and A 2\nExplanation:\nThis first number goes until position D, whereas the second goes until position A. The first number is the longest, so we sum digits in positions A, B, C, D.\nWe sum the digits in position A, which are A 2 and A 2. That is 2 + 2, which equals to 4. Because 4 is less than or equal to 9, this results in carry 0 and A 4.\nThis carry 0 is summed with digits in position B, which are B 6 and B 0. That is 0 + 6 + 0, which equals to 6. Because 6 is less than or equal to 9, this results in carry 0 and B 6.\nThis carry 1 is summed with digits in position C, which are C 0 and C 0. That is 0 + 0 + 0, which equals to 0. Because 0 is less than or equal to 9, this results in carry 0 and C 0.\nThis carry 0 is summed with digits in position D, which are D 8 and D 0. That is 0 + 8 + 0, which equals to 8. Because 8 is less than or equal to 9, this results in carry 0 and D 8.\nOutput: D 8 C 0 B 6 A 4\n\n"

In [20]:
print(base_prompt)

Example 1:
Instruction: add the numbers E 4 D 1 C 6 B 5 A 4 and D 1 C 8 B 5 A 3   
Explanation:
This first number goes until position E, whereas the second goes until position D. The first number is the longest, so we sum digits in positions A, B, C, D, E.
We sum the digits in position A, which are A 4 and A 3. That is 4 + 3, which equals to 7. Because 7 is less than or equal to 9, this results in carry 0 and A 7.
This carry 0 is summed with digits in position B, which are B 5 and B 5. That is 0 + 5 + 5, which equals to 10. Because 1 0 is greater than 9, this results in carry 1 and B 0.
This carry 1 is summed with digits in position C, which are C 6 and C 8. That is 1 + 6 + 8, which equals to 15. Because 1 5 is greater than 9, this results in carry 1 and C 5.
This carry 1 is summed with digits in position D, which are D 1 and D 1 .That is 1 + 1 + 1, which equals to 3. Because 3 is less or equal than 9, this results in carry 0 and D 3.
This carry 0 is summed with digits in position E, w

In [21]:
test_samples = pd.read_csv(DATA_PATH, sep='\t')

In [22]:
test_samples

,instruction,output
0,add the numbers L 2 K 7 J 0 I 8 H 5 G 1 F 5 E ...,271474905549
1,add the numbers M 9 L 7 K 7 J 4 I 0 H 5 G 3 F ...,9774053757802
2,add the numbers G 3 F 8 E 6 D 6 C 3 B 8 A 8 an...,3927672
3,add the numbers D 4 C 6 B 3 A 4 and D 1 C 1 B ...,5814
4,add the numbers E 4 D 3 C 7 B 0 A 3 and D 1 C ...,45550
...,...,...
195,add the numbers E 9 D 2 C 3 B 5 A 1 and E 3 D ...,126029
196,add the numbers F 7 E 3 D 3 C 8 B 0 A 2 and K ...,21198463346
197,add the numbers G 7 F 0 E 3 D 3 C 5 B 4 A 7 an...,7038303
198,add the numbers E 7 D 5 C 4 B 2 A 9 and D 9 C ...,85276


In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
responses = []

for idx, sample in enumerate(test_samples['instruction']):

  prompt_sample=base_prompt+'Example 6:\nInstruction: '+sample+'\nExplanation:'

  response = openai.Completion.create(
  engine="text-davinci-002",
  prompt=prompt_sample,
  temperature=0,
  max_tokens=1024,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["\n\n"]
  )
  responses.append(response)

In [26]:
responses

[<OpenAIObject text_completion id=cmpl-572iHLs9xQ1tzZaGG1ZW5AJ0l5MXE at 0x7f3204800110> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nThis first number goes until position L, whereas the second goes until position I. The first number is the longest, so we sum digits in positions A, B, C, D, E, F, G, H, I, J, K, L.\nWe sum the digits in position A, which are A 9 and A 0. That is 9 + 0, which equals to 9. Because 9 is less than or equal to 9, this results in carry 0 and A 9.\nThis carry 0 is summed with digits in position B, which are B 4 and B 0. That is 0 + 4 + 0, which equals to 4. Because 4 is less than or equal to 9, this results in carry 0 and B 4.\nThis carry 0 is summed with digits in position C, which are C 2 and C 3. That is 0 + 2 + 3, which equals to 5. Because 5 is less than or equal to 9, this results in carry 0 and C 5.\nThis carry 0 is summed with digits in position D, which are D 8 and D 7. That is 0

In [28]:
!pip uninstall pandas
!pip install pandas

Found existing installation: pandas 1.3.5
Uninstalling pandas-1.3.5:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/pandas/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/config.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/dates.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/display.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/localization.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/indexing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/interval.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/missing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/pr

In [29]:
#save results
prompt = test_samples['instruction']
ground_truth=test_samples['output']
save_results(prompt, ground_truth, responses, SAVE_PATH)